<a href="https://colab.research.google.com/github/CardiacModelling/summer-school-2025/blob/main/Hands_on_session_2_action_potential_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands-on session 2: action potential models

* Downloading a CellML and loading into myokit
* Set up steady pacing experiment
* Change the frequency
* Have a look at APs, plot the now shorter/longer ones
* Calculate APDs
* Stick in a loop to draw a steady-state restitution curve.
* Apply simulated drug block or mutation by scaling a conductance and seeing how things change.
* Nice to show EAD or something happening, to link to arrhythmic whole-cell ideas (but let others do that).

## Run this first:

In [2]:
# Don't show the installation logs
%%capture
# If installation fails, comment out the line above and run again!

# Install Myokit and Sundials
!apt-get install libsundials-dev
!pip install myokit

# Disable debug messages annoyingly shown on colab
import logging
logging.disable(logging.INFO)

# Import Myokit and matplotlib
import myokit
import matplotlib

## Downloading a CellML and loading into myokit


## Set up steady pacing experiment

## Change the frequency

## Have a look at APs, plot the now shorter/longer ones

## Calculate APDs


## Stick in a loop to draw a steady-state restitution curve.

## Apply simulated drug block or mutation by scaling a conductance and seeing how things change.


## Nice to show EAD or something happening, to link to arrhythmic whole-cell ideas (but let others do that).